<a href="https://colab.research.google.com/github/dovele/scraper/blob/main/ebay_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Scraping eBay

In [ ]:
def scrape_data(items_to_scrape, keywords):
  """
  Scrapes ebay website for a number of samples for each keyword
  
  :param items_to_scrape: integer of items to scrape for each keyword.
  :param keywords: list of keywords to scrape.
  :return: pandas dataframe with the following columns:
    Category (keyword), title, price, item url, image url.
    
  """

  data = {"category": [], "item_title": [], "item_price": [], "item_url": [], "item_image": []}
  for keyword in keywords:
    page_url = []
    for i in range(1,round((items_to_scrape/203) + 1)):
        page_url.append('https://www.ebay.com/sch/i.html?_from=R40&_nkw=' + keyword + '&_sacat=0&_ipg=192&_pgn=' + str(i))
    # details of the info from the website
    for links in page_url:
      print(links)
      response = requests.get(links)
      soup = BeautifulSoup(response.content, 'html.parser')

      
      for title in soup.find_all('h3', { 'class': 's-item__title' }):
        data["item_title"].append(title.text)
        data["category"].append(keyword)
      for price in soup.find_all('span', { 'class':"s-item__price" }):
        data["item_price"].append(price.text)
      for url_of_item in soup.find_all('a', { 'class': 's-item__link' }):
        data["item_url"].append(url_of_item.get('href'))
      for url_of_image in soup.find_all('img', { 'class': 's-item__image-img' }):
        data["item_image"].append(url_of_image['src'])
    page_url.clear()

  df = pd.DataFrame.from_dict(data, orient='index')
  return df.transpose()

In [ ]:
 scrape_data(3000, ['dress', 'bikini', 'sunglasses']) 

https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=1
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=2
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=3
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=4
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=5
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=6
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=7
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=8
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=9
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=10
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=11
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=12
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0

,category,item_title,item_price,item_url,item_image
0,dress,,$35.74,https://www.ebay.com/itm/01920391?epid=2603981...,https://i.ebayimg.com/thumbs/images/g/mpYAAOSw...
1,dress,Nicole Miller Artelier VERYBERRY/CHERRY Color ...,$14.99,https://www.ebay.com/itm/114588060630?_trkparm...,https://i.ebayimg.com/thumbs/images/g/mdMAAOSw...
2,dress,Women Ultra High Slit Legs Nightgown Dress Lin...,$15.97,https://www.ebay.com/itm/362851067149?_trkparm...,https://i.ebayimg.com/thumbs/images/g/CrIAAOSw...
3,dress,Womens Casual Tunic Solid Long Sleeve Maxi Dre...,$5.33,https://www.ebay.com/itm/284380707602?_trkparm...,https://i.ebayimg.com/thumbs/images/g/uH8AAOSw...
4,dress,Kaftan Tunic Kimono Dress Ladies Summer Women ...,$6.93,https://www.ebay.com/itm/363500570835?_trkparm...,https://i.ebayimg.com/thumbs/images/g/GfMAAOSw...
...,...,...,...,...,...
2501,None,None,$125.00,None,None
2502,None,None,$298.73,None,None
2503,None,None,$300.04,None,None
2504,None,None,$139.99,None,None


# Connecting to Heroku and inserting scraped data into tables

In [ ]:
import psycopg2
import pandas as pd

def connect_database():
    """
    Connection to work with the remote database on Heroku platform.
    :return: connection
    """
    connection = psycopg2.connect(
      database="d9peu2e501rl36",
      user="ixlplailywcloj",
      password="be110b709e1f619238f50e817038e07b875dcdd90bbb631096ab3f5ba101390e",
      host="ec2-176-34-222-188.eu-west-1.compute.amazonaws.com",
      port="5432"
  )

    return connection

In [ ]:
def create_insert_table(df):
    """
    Create tables and insert dataframe in database.
    :return: None
    """
    connection = connect_database()
    cur = connection.cursor()

    cur.execute('''
        CREATE TABLE IF NOT EXISTS categories (
            id serial PRIMARY KEY,
            category VARCHAR(50)
        );
        ''')

    cur.execute('''
        CREATE TABLE IF NOT EXISTS ebay (
        id serial PRIMARY KEY,
        item_title varchar(10000),
        item_price varchar(255),
        item_url varchar(10000),
        item_image varchar(10000),
        category varchar(255)
        );
        ''')

    #Get array of unique category names
    unique_categories = df['category'].str.split(',').explode().unique().tolist()
    cat = [i for i in range(1, len(unique_categories)+1)]
    # Insert unique category names to the categories table
    for i in cat:
        cur.execute(f"INSERT INTO categories (category) VALUES ('{i}');")

    # insert data df into ebay table
    for index, row in df.iterrows():
            cur.execute(
                "INSERT INTO ebay (item_title, item_price, item_url, item_image, category) values(%s, %s, %s, %s, %s)",
                (row.item_title, row.item_price, row.item_url, row.item_image, row.category),
            )

    connection.commit()

def join_and_export():
    """
    Execute query, fetch all the records and export it to CSV file.
    :return: CSV file.
    """
    connection = connect_database()
    cur = connection.cursor()

    cur.execute(
            "select categories.category, ebay.item_title, ebay.item_price, ebay.item_url, ebay.item_image from ebay LEFT JOIN categories on ebay.category = categories.category"
        )

    df = pd.DataFrame(cur.fetchall())
    df.to_csv('data.csv')

In [ ]:
create_insert_table(scrape_data(3000, ['dress', 'bikini', 'sunglasses']))

https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=1
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=2
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=3
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=4
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=5
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=6
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=7
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=8
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=9
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=10
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=11
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0&_ipg=192&_pgn=12
https://www.ebay.com/sch/i.html?_from=R40&_nkw=dress&_sacat=0

In [ ]:
#Screenshot of a table in Heroku database
![](https://drive.google.com/uc?export=view&id=1MurExPECblhA5uABvpEZFXL44ky7reMw)

/bin/bash: -c: line 0: syntax error near unexpected token `https://drive.google.com/uc?export=view'
/bin/bash: -c: line 0: `[](https://drive.google.com/uc?export=view&id=1MurExPECblhA5uABvpEZFXL44ky7reMw)'
